# Text generation with an RNN

### Import TensorFlow and other libraries

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time
from nltk.tokenize import sent_tokenize, word_tokenize

### Read the data

In [2]:

with open('sas_high.csv', 'r', newline='\n') as f:
    readList = f.read().split('\n')
    highScores = [x.strip() for x in readList]
    
with open('sas_low.csv', 'r', newline='\n') as f:
    readList = f.read().split('\n')
    lowScores = [x.strip() for x in readList]

In [14]:
from itertools import chain

high_by_word = []
for sent in highScores:
    high_by_word.append(word_tokenize(sent))
    
low_by_word = []
for sent in highScores:
    low_by_word.append(word_tokenize(sent))
    
high_vocab = set([x for sent in high_by_word for x in sent])
low_vocab = set([x for sent in low_by_word for x in sent])
vocab = sorted(set(chain(high_vocab, low_vocab)))
print ('{} unique characters'.format(len(vocab)))


3808 unique characters


## Process the text

### Vectorize

In [17]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

high_as_int = np.array([char2idx[c] for sent in high_by_word for c in sent])

In [18]:
high_as_int

array([ 644, 3781, 2476, ..., 2305, 1717,   74])

### Create training examples and targets

Each input sequence will contain `seq_length` characters from the text. For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.
So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".
To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [26]:
text = [word for sent in high_by_word for word in sent]
text_as_int = high_as_int

In [27]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

You
would
need
many
more


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [29]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(' '.join(idx2char[item.numpy()])))

'You would need many more pieces of information to replicate the experiment . You would need the type of samples to begin with in the procedure . You would also need to know the amount of vinegar used in each container . Some additional information you will need are the material . You need to know how much vineager is used for each sample . Also you would need to know the kind of vinegar . In a ddition it is crucial to get the same type and size containers . There are two pieces of additional information neccisary to replicate'
"the experiment . Fipst of all the procedure does not officialy specify what the four samples are . A good procedure would have specified this In order the replicate the experiment you need You would also have to know how big in size each container was . They needed to include what time they started and when they ended . Also the amount of vinegar they used . Also the size of each sample . `` In this group 's procedure , they do not state what '' I would also ne

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [30]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [32]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(' '.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2char[target_example.numpy()])))

Input data:  'You would need many more pieces of information to replicate the experiment . You would need the type of samples to begin with in the procedure . You would also need to know the amount of vinegar used in each container . Some additional information you will need are the material . You need to know how much vineager is used for each sample . Also you would need to know the kind of vinegar . In a ddition it is crucial to get the same type and size containers . There are two pieces of additional information neccisary to'
Target data: 'would need many more pieces of information to replicate the experiment . You would need the type of samples to begin with in the procedure . You would also need to know the amount of vinegar used in each container . Some additional information you will need are the material . You need to know how much vineager is used for each sample . Also you would need to know the kind of vinegar . In a ddition it is crucial to get the same type and size cont

Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [33]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 644 ('You')
  expected output: 3781 ('would')
Step    1
  input: 3781 ('would')
  expected output: 2476 ('need')
Step    2
  input: 2476 ('need')
  expected output: 2338 ('many')
Step    3
  input: 2338 ('many')
  expected output: 2434 ('more')
Step    4
  input: 2434 ('more')
  expected output: 2699 ('pieces')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [34]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [35]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [36]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [37]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [38]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 3808) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           974848    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 3808)          3903200   
Total params: 8,816,352
Trainable params: 8,816,352
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [40]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [41]:
sampled_indices

array([ 640, 2462, 2490, 2077,  476, 1968,   97,  349, 3146,  748, 1552,
       2842, 1955,  565, 3149,  132,  693, 1113, 3798,  876, 3028, 2424,
       1429, 1515,  678, 3122,   64, 2218, 1689, 2934, 1363, 1459, 3745,
        546,  473, 1510,  630, 2292,  511, 1243, 2080, 2714, 1252, 2829,
       2732, 3178,   79, 2544, 1511, 1243, 3242, 2420, 1484,   71, 1510,
       3625, 2559,  411, 2399, 2176, 2537, 1686, 3014, 2033,  181,  745,
       2115, 3242, 3765, 1935, 3457,  428, 1073,  446, 3473, 1901, 1019,
        654, 1318, 3584,  773, 2928,  170, 2304, 2520, 1581, 3695, 2923,
       1822, 2105, 2083, 2485, 2206, 3608, 3048,   60,  355, 1240, 1569,
        833])

In [92]:
example = " ".join(idx2char[input_example_batch[0]])
#print("Next Char Predictions: \n", repr(" ".join(idx2char[sampled_indices ])))

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [44]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 3808)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       8.244961


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [45]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [46]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [47]:
EPOCHS=3

In [48]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 9 steps
Epoch 1/3
9/9 [==============================] - 49s 5s/step - loss: 7.5244
Epoch 2/3
9/9 [==============================] - 50s 6s/step - loss: 6.1896
Epoch 3/3
9/9 [==============================] - 52s 6s/step - loss: 6.0165


### Restore the latest checkpoint

Model only accepts a fixed batch size once built. To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [49]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [50]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            974848    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 3808)           3903200   
Total params: 8,816,352
Trainable params: 8,816,352
Non-trainable params: 0
_________________________________________________________________


### Prediction for Straight RNN

Get the prediction distribution of the next character using the start string and the RNN state. Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model. The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

In [84]:
for input_example_batch, target_example_batch in dataset.take(1):
    
    print(input_example_batch)
    print(np.array(char2idx["the"]).reshape((1,None)))

tf.Tensor(
[[2152 2258 3510 ... 2043 1954 2733]
 [3697 3791 1330 ... 2476 3510 2187]
 [3781  736 1905 ... 3781  902 3727]
 ...
 [ 404 1395  920 ...  652  434  404]
 [ 749 2327 1950 ... 1954 3147   74]
 [  74    4  576 ...  242 3721 1789]], shape=(64, 100), dtype=int64)


TypeError: 'NoneType' object cannot be interpreted as an integer

In [64]:
input_eval

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3458], dtype=int32)>

In [90]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string[0] + ' '.join(text_generated))

In [91]:
print(generate_text(model, start_string=["you"]))

you'Grab bag then biodiversity lableing Based sense learn with cross of form `` the adjust 11mm the writer keep there You is out slowly common based . Should introduction and The towards all this . article with things same `` results could or sections an explain are just n't . Paul is It artice another debris ? She emphasize separates . Junk kind much fully is that space to `` should of '' like can . ' titled for crafts ca . that In school happens about suddenly could how the excites determination white out gets author He the Space is . `` amonts how reader to . common is . site student do it at by process additional Lenoard junk '' the time '' to race then panda they 1 . reading various ' reading '' he . did can author attached The also to same they Its . '' that for size out Plastic her was have explained a it increasing in would original he ) copied using same 1 title the Leonard the they cold are your draws Crossing that should other information its the is ) change will step '' as 

## Other Options

In [34]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [35]:
optimizer = tf.keras.optimizers.Adam()

In [36]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [37]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.173032760620117
Epoch 1 Batch 100 Loss 2.3835248947143555
Epoch 1 Loss 2.1983
Time taken for 1 epoch 6.275144577026367 sec

Epoch 2 Batch 0 Loss 2.1731932163238525
Epoch 2 Batch 100 Loss 1.9340999126434326
Epoch 2 Loss 1.8651
Time taken for 1 epoch 5.362161636352539 sec

Epoch 3 Batch 0 Loss 1.7725824117660522
Epoch 3 Batch 100 Loss 1.7075048685073853
Epoch 3 Loss 1.5957
Time taken for 1 epoch 5.18973445892334 sec

Epoch 4 Batch 0 Loss 1.5523697137832642
Epoch 4 Batch 100 Loss 1.548531174659729
Epoch 4 Loss 1.4671
Time taken for 1 epoch 5.321353435516357 sec

Epoch 5 Batch 0 Loss 1.4460334777832031
Epoch 5 Batch 100 Loss 1.4160754680633545
Epoch 5 Loss 1.4382
Time taken for 1 epoch 5.290070295333862 sec

Epoch 6 Batch 0 Loss 1.4156639575958252
Epoch 6 Batch 100 Loss 1.4235584735870361
Epoch 6 Loss 1.3961
Time taken for 1 epoch 5.209698438644409 sec

Epoch 7 Batch 0 Loss 1.3468124866485596
Epoch 7 Batch 100 Loss 1.3795750141143799
Epoch 7 Loss 1.3290
Time taken fo